In [1]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np

In [2]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': 'CfScAOZ6JczBx2FCuJse8nARNyA7crnaeaWIL0dmp2JMzXYsJ9_RUpqK6WTz5_nnEqcjIy0Qc34z3NBEjHwTLfonWo6tQJ2zcTSZIORfvyWS7Q3Fgoq9vqv4q7mxcFOTnf6u_u0HQYtPW8zMtkq-e4MIggWgWPwyns8pfLOrCDJk8GDNny8YOEM3czJkivZkLxNrZNPROPP9W81zBOnRoK2WlhjRP4iA8GB3aY14cPlx_jRLZPnSFO-MmrjG2TttiphW-OZfakJuYNCdm2m1YNbtN3qObM63Y61r39C0P1as5LT0fMnZiTW0dyb5oIa4RGmqtPUbHCnkn76TghioR6zvS4A',
 'current_data': (datetime.datetime(2022, 9, 16, 14, 11, 19, 676886, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                      timeStamp    value    fuelType
  0   2022-09-16 00:00:00-06:00  1264.00        Coal
  1   2022-09-16 00:05:00-06:00  1252.00        Coal
  2   2022-09-16 00:10:00-06:00  1256.00        Coal
  3   2022-09-16 00:15:00-06:00  1259.00        Coal
  4   2022-09-16 00:20:00-06:00  1264.00        Coal
  ..                        ...      ...         ...
  165 2022-09-16 13:45:00-06:00    95.99  Pool Price
  166 2022-09-16 13:50:00-06:00    95.99  Pool Price
  167 2022-09-16 13:55:

In [16]:
[t for t,df in default_pickle['monthly_outage_dfs']]

[datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 12, 14, 0, 34, 184434, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
 datetime.datetime(2022, 9, 16, 13, 14, 11, 608997, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>)]

In [33]:
default_pickle['daily_outage_dfs'][6][1]['fuelType'].unique()

array(['Intertie', 'Biomass & Other', 'Coal', 'Dual Fuel',
       'Energy Storage', 'Hydro', 'Natural Gas', 'Solar', 'Wind',
       '7-Day Wind Forecast', '3-Day Solar Forecast'], dtype=object)

In [45]:
default_pickle['daily_outage_dfs'][6][1][default_pickle['daily_outage_dfs'][6][1]['fuelType'] != ('3-Day Solar Forecast' or '7-Day Wind Forecast')]

,timeStamp,value,fuelType
0,2022-09-16 00:00:00-06:00,449.00,Intertie
1,2022-09-16 01:00:00-06:00,0.00,Intertie
2,2022-09-16 02:00:00-06:00,0.00,Intertie
3,2022-09-16 03:00:00-06:00,257.00,Intertie
4,2022-09-16 04:00:00-06:00,361.00,Intertie
...,...,...,...
163,2022-09-22 19:00:00-06:00,690.92,7-Day Wind Forecast
164,2022-09-22 20:00:00-06:00,715.13,7-Day Wind Forecast
165,2022-09-22 21:00:00-06:00,748.93,7-Day Wind Forecast
166,2022-09-22 22:00:00-06:00,778.22,7-Day Wind Forecast


In [59]:
default_pickle['daily_outage_dfs'][6][1]


,timeStamp,value,fuelType
0,2022-09-16 00:00:00-06:00,449.00,Intertie
1,2022-09-16 01:00:00-06:00,0.00,Intertie
2,2022-09-16 02:00:00-06:00,0.00,Intertie
3,2022-09-16 03:00:00-06:00,257.00,Intertie
4,2022-09-16 04:00:00-06:00,361.00,Intertie
...,...,...,...
65,2022-09-18 17:00:00-06:00,514.70,3-Day Solar Forecast
66,2022-09-18 18:00:00-06:00,368.67,3-Day Solar Forecast
67,2022-09-18 19:00:00-06:00,173.12,3-Day Solar Forecast
68,2022-09-18 20:00:00-06:00,15.13,3-Day Solar Forecast
